In [187]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import os

from skforecast.model_selection import random_search_forecaster
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.ensemble import RandomForestRegressor

In [214]:
dados = pd.DataFrame(columns=['ref_bacen', 'date', 'indice'])

diretorio = '../data/rl/'

arquivos_json = [arquivo for arquivo in os.listpredicoes.dir(diretorio) if arquivo.endswith('.json')]

for arquivo_json in arquivos_json:
    with open(os.path.join(diretorio, arquivo_json), 'r') as file:
        d = json.loads(file.read())
        
        data_temp = pd.DataFrame({
            'ref_bacen': arquivo_json,
            'date': d['Indices']['NDVI']['Serie Processada']['Data'],
            'indice': d['Indices']['NDVI']['Serie Processada']['Indice']
        })

        dados = pd.concat([dados, data_temp], ignore_index=True)

dados['ref_bacen'] = dados['ref_bacen'].str.split('_').str.get(0)

dados['date'] = pd.to_datetime(dados['date'], format='%Y-%m-%d')
# dados = dados.set_index('date')


,ref_bacen,date,indice
31,st,2020-08-12,0.010591
30,st,2020-07-03,0.021134
29,st,2020-06-09,0.034286
28,st,2020-04-14,0.092785
27,st,2020-03-29,0.018291
...,...,...,...
11,505954021,2017-07-28,0.000657
124,505967820,2017-07-20,0.022933
53,505967820,2017-07-20,0.022933
112,505971238,2017-07-12,0.062466


In [215]:
dados[dados['ref_bacen'] == '506273616']

,ref_bacen,date,indice
0,506273616,2017-08-21,0.018645
1,506273616,2017-10-16,0.024464
2,506273616,2017-11-09,0.015563
3,506273616,2017-11-25,0.003720
4,506273616,2017-12-11,0.005570
5,506273616,2017-12-19,0.007996
6,506273616,2018-01-09,0.008844
7,506273616,2018-02-18,0.010653
8,506273616,2018-02-26,0.015367
9,506273616,2018-07-12,0.045398


Model validation    

In [190]:
previsoes = []

for ref_bacen in dados['ref_bacen'].unique():
    dados_ref = dados[dados['ref_bacen'] == ref_bacen]['indice']

    modelo = SARIMAX(dados_ref, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    resultado = modelo.fit()

    previsao = resultado.get_forecast(steps=6)

    datas_previsao = pd.date_range(start=dados.index[-1] + pd.DateOffset(1), periods=6, freq='M')

    previsao_mean = previsao.predicted_mean
    previsao_conf_int = previsao.conf_int()

    
    previsao_df = pd.DataFrame({
        'date': datas_previsao,
        'ref_bacen': ref_bacen,
        'indice_inicial': dados_ref.iloc[-1],  
        'previsao': previsao_mean,
        'lower_conf_int': previsao_conf_int.iloc[:, 0],
        'upper_conf_int': previsao_conf_int.iloc[:, 1]
    })

    previsoes.append(previsao_df)

previsoes_df = pd.concat(previsoes)

/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/statespa

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -0.00000D+00    |proj g|=  0.00000D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5      0      1      0     0     0   0.000D+00  -0.000D+00
  F =  -0.0000000000000000     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -1.1

/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index h

At iterate   20    f= -1.26524D+00    |proj g|=  2.64481D-01

At iterate   25    f= -1.26531D+00    |proj g|=  4.51000D-02

At iterate   30    f= -1.26568D+00    |proj g|=  2.59272D-01

At iterate   35    f= -1.26638D+00    |proj g|=  2.79847D-01

At iterate   40    f= -1.27522D+00    |proj g|=  8.91499D-02

At iterate   45    f= -1.27602D+00    |proj g|=  1.26931D-01

At iterate   50    f= -1.27603D+00    |proj g|=  2.29550D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     50     81      1     0     0   2.296D-02  -1.276D+00
  F =  -1.2760329090324642     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT        

/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has

...


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No su


           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     46    160      3     2     0   7.182D-02  -1.849D+00
  F =  -1.8493683598272073     

ABNORMAL_TERMINATION_IN_LNSRCH                              
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -2.61394D+00    |proj g|=  2.10626D+05
  ys=-2.943E+00  -gs= 4.023E-01 BFGS update SKIPPED
  ys=-1.600E+01  -gs= 5.568E-01 BFGS update SKIPPED
  ys=-8.552E-04  -gs= 1.182E-01 BFGS update SKIPPED

           * * *

Tit   = total numbe


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No su

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -2.61394D+00    |proj g|=  2.10626D+05
  ys=-2.943E+00  -gs= 4.023E-01 BFGS update SKIPPED
  ys=-1.600E+01  -gs= 5.568E-01 BFGS update SKIPPED
  ys=-8.552E-04  -gs= 1.182E-01 BFGS update SKIPPED

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5      5     81      2     3     0   2.391D+05  -4.476D+00
  F =  -4.4760050460575025     

ABNORMAL_TERMINATION_IN_LNSRCH                              



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/lguirald/git/personal/1-environments/api/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No su

In [191]:
previsoes_df

,date,ref_bacen,indice_inicial,previsao,lower_conf_int,upper_conf_int
11,2018-07-31,506273616,0.000964,0.000964,-1959.964001,1959.965928
12,2018-08-31,506273616,0.000964,0.010286,-2400.447352,2400.467925
13,2018-09-30,506273616,0.000964,0.016105,-2400.442334,2400.474544
14,2018-10-31,506273616,0.000964,0.007204,-2400.452035,2400.466443
15,2018-11-30,506273616,0.000964,-0.004639,-2400.464678,2400.455400
16,2018-12-31,506273616,0.000964,-0.002789,-2400.463628,2400.458050
34,2018-07-31,505954021,0.002372,0.010658,-0.041582,0.062899
35,2018-08-31,505954021,0.002372,-0.000790,-0.053280,0.051701
36,2018-09-30,505954021,0.002372,0.006574,-0.044794,0.057942
37,2018-10-31,505954021,0.002372,0.011403,-0.040036,0.062842


In [192]:

# Filtrar os dados para um 'ref_bacen' específico, se necessário
ref_bacen = '505993977'  # Substitua pelo valor desejado

dados_filtrados = previsoes_df[previsoes_df['ref_bacen'] == ref_bacen]


,date,ref_bacen,indice_inicial,previsao,lower_conf_int,upper_conf_int
32,2018-07-31,505993977,0.00725,0.007400,-0.012323,0.027123
33,2018-08-31,505993977,0.00725,0.015799,-0.003918,0.035517
34,2018-09-30,505993977,0.00725,0.007220,-0.012546,0.026985
35,2018-10-31,505993977,0.00725,0.005008,-0.014797,0.024813
36,2018-11-30,505993977,0.00725,0.005784,-0.014064,0.025632
37,2018-12-31,505993977,0.00725,0.013813,-0.006079,0.033705


In [204]:
previsoes_df_final = previsoes_df[['date', 'ref_bacen', 'previsao']][previsoes_df['ref_bacen']!='st']

In [208]:
previsoes_df_final['ref_bacen'].unique()

array(['506273616', '505954021', '505993977', '505967820', '505971238'],
      dtype=object)

In [210]:
previsoes_df_final.to_csv('../data/previsoes.csv')